In [ ]:
import cv2
import numpy as np
import math
cap = cv2.VideoCapture(0)
     
while(1):
        
    try:  #an error comes if it does not find anything in window as it cannot find contour of max area
          #therefore this try error statement
          
        ret, frame = cap.read()
        frame=cv2.flip(frame,1)
        kernel = np.ones((3,3),np.uint8)
        
        #define region of interest
        roi=frame[100:300, 100:300]
        
        
        cv2.rectangle(frame,(100,100),(300,300),(0,255,0),0)    
        hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
        
        
         
    # define range of skin color in HSV
        lower_skin = np.array([0,20,70], dtype=np.uint8)
        upper_skin = np.array([20,255,255], dtype=np.uint8)
        
     #extract skin colur imagw  
        mask = cv2.inRange(hsv, lower_skin, upper_skin)
        
   
        
    #extrapolate the hand to fill dark spots within
        mask = cv2.dilate(mask,kernel,iterations = 4)
        
    #blur the image
        mask = cv2.GaussianBlur(mask,(5,5),100) 
        
        
        
    #find contours
        _,contours,hierarchy= cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
   #find contour of max area(hand)
        cnt = max(contours, key = lambda x: cv2.contourArea(x))
        
    #approx the contour a little
        epsilon = 0.0005*cv2.arcLength(cnt,True)
        approx= cv2.approxPolyDP(cnt,epsilon,True)
       
        
    #make convex hull around hand
        hull = cv2.convexHull(cnt)
        
     #define area of hull and area of hand
        areahull = cv2.contourArea(hull)
        areacnt = cv2.contourArea(cnt)
      
    #find the percentage of area not covered by hand in convex hull
        arearatio=((areahull-areacnt)/areacnt)*100
    
     #find the defects in convex hull with respect to hand
        hull = cv2.convexHull(approx, returnPoints=False)
        defects = cv2.convexityDefects(approx, hull)
        
    # l = no. of defects
        l=0
        
    #code for finding no. of defects due to fingers
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(approx[s][0])
            end = tuple(approx[e][0])
            far = tuple(approx[f][0])
            pt= (100,180)
            
            
            # find length of all sides of triangle
            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            s = (a+b+c)/2
            ar = math.sqrt(s*(s-a)*(s-b)*(s-c))
            
            #distance between point and convex hull
            d=(2*ar)/a
            
            # apply cosine rule here
            angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57
            
        
            # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
            if angle <= 90 and d>30:
                l += 1
                cv2.circle(roi, far, 3, [255,0,0], -1)
            
            #draw lines around hand
            cv2.line(roi,start, end, [0,255,0], 2)
            
            
        l+=1
        
        #print corresponding gestures which are in their ranges
        font = cv2.FONT_HERSHEY_SIMPLEX
        if l==1:
            if areacnt<2000:
                cv2.putText(frame,'Put hand in the box',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            else:
                if arearatio<12:
                    cv2.putText(frame,'0',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                elif arearatio<17.5:
                    cv2.putText(frame,'Best of luck',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                   
                else:
                    cv2.putText(frame,'1',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    
        elif l==2:
            cv2.putText(frame,'2',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        elif l==3:
         
              if arearatio<27:
                    cv2.putText(frame,'3',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
              else:
                    cv2.putText(frame,'ok',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
                    
        elif l==4:
            cv2.putText(frame,'4',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        elif l==5:
            cv2.putText(frame,'5',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        elif l==6:
            cv2.putText(frame,'reposition',(0,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        else :
            cv2.putText(frame,'reposition',(10,50), font, 2, (0,0,255), 3, cv2.LINE_AA)
            
        #show the windows
        cv2.imshow('mask',mask)
        cv2.imshow('frame',frame)
    except:
        pass
        
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break
    
cv2.destroyAllWindows()
cap.release()

In [9]:
def f(a=None,b=None,c=None):
    print(a,b)

In [10]:
f(a= 1, b=2)

1 2


In [11]:
import numpy as np
import pandas as pd

In [12]:
data = pd.read_csv("Test1.csv")

In [13]:
data

,RNCID,Address,city,state,zip,zip4,id
0,14962562320,807 ROBERTSON ST,BROOKHAVEN,MS,39601.0,2535.0,1407
1,14962562322,245 MEGAN LN,COLUMBUS,MS,39705.0,3193.0,1408
2,14962562324,607 S COLLEGE ST,BRANDON,MS,39042.0,3313.0,1409
3,14962562328,18119 S VILLAGE DR,SAUCIER,MS,39574.0,6521.0,1410
4,14962562329,9570 LAA LA WAY,DIAMONDHEAD,MS,39525.0,4182.0,1411
...,...,...,...,...,...,...,...
942,14962565321,121 GRAYHAWK DR,MADISON,MS,39110.0,4004.0,2349
943,14962565324,872 BETHEL CHURCH RD,SUMRALL,MS,39482.0,4131.0,2350
944,14962565326,3675 ATTALA ROAD 3007,WEST,MS,39192.0,7723.0,2351
945,14962565331,606 HERRING CIR,TUPELO,MS,38804.0,5016.0,2352


In [15]:
type(np.array_split(data, 4))

list

In [16]:
arr = np.array_split(data, 4)

In [18]:
arr[0]

,RNCID,Address,city,state,zip,zip4,id
0,14962562320,807 ROBERTSON ST,BROOKHAVEN,MS,39601.0,2535.0,1407
1,14962562322,245 MEGAN LN,COLUMBUS,MS,39705.0,3193.0,1408
2,14962562324,607 S COLLEGE ST,BRANDON,MS,39042.0,3313.0,1409
3,14962562328,18119 S VILLAGE DR,SAUCIER,MS,39574.0,6521.0,1410
4,14962562329,9570 LAA LA WAY,DIAMONDHEAD,MS,39525.0,4182.0,1411
...,...,...,...,...,...,...,...
232,14962563039,1318 OLD HIGHWAY 35 S,FOXWORTH,MS,39483.0,4032.0,1639
233,14962563040,13725 HIGHWAY 6,THAXTON,MS,38871.0,8703.0,1640
234,14962563045,62833 HWY 25 N,AMORY,MS,38821.0,NaN,1641
235,14962563047,687 THORNTON RD,SCOOBA,MS,39358.0,7258.0,1642


In [19]:
arr[1]

,RNCID,Address,city,state,zip,zip4,id
237,14962563061,101 CAROL ST,DURANT,MS,39063.0,NaN,1644
238,14962563064,116 ARTHUR CT,GULFPORT,MS,39503.0,3338.0,1645
239,14962563070,301 DOGWOOD CIR,NEW AUGUSTA,MS,39462.0,9103.0,1646
240,14962563072,10500 ROAD 4316,COLLINSVILLE,MS,39325.0,NaN,1647
241,14962563075,1037 WEST ST,WESSON,MS,39191.0,7714.0,1648
...,...,...,...,...,...,...,...
469,14962563767,155 HARPER ST,RIDGELAND,MS,39157.0,8675.0,1876
470,14962563770,231 SUNRISE DR,TUPELO,MS,38801.0,7752.0,1877
471,14962563774,233 FIELDSTONE LN,MADISON,MS,39110.0,5051.0,1878
472,14962563776,426 LAROW D,CARRIERE,MS,39426.0,8456.0,1879


In [20]:
arr[2]

,RNCID,Address,city,state,zip,zip4,id
474,14962563780,410 NINTH ST,LAMBERT,MS,38643.0,NaN,1881
475,14962563781,2902 S LIBERTY ST,CANTON,MS,39046.0,9210.0,1882
476,14962563786,610 W MAIN ST,FULTON,MS,38843.0,1024.0,1883
477,14962563787,406 CHAPIN ST,STARKVILLE,MS,39759.0,2683.0,1884
478,14962563794,869 N DEERFIELD DR,CANTON,MS,39046.0,9286.0,1885
...,...,...,...,...,...,...,...
706,14962564604,9 W WILDERNESS RD,NATCHEZ,MS,39120.0,8827.0,2113
707,14962564605,6057 OLD HILL RD,BAY SAINT LOUIS,MS,39520.0,8005.0,2114
708,14962564609,1120 E LEE BLVD P161,STARKVILLE,MS,39759.0,8751.0,2115
709,14962564610,410 OLD WESSON RD NE APT 3,BROOKHAVEN,MS,39601.0,2890.0,2116


In [22]:
arr[3].to_csv("abc.csv")